In [0]:
try: 
  %tensorflow_version 2.x
except Exception:
  pass

import tensorflow as tf

from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras import Model

데이터셋을 로드하여 준비합니다.


In [0]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

# 채널 차원을 추가합니다.
x_train = x_train[..., tf.newaxis]
x_test = x_test[..., tf.newaxis]

y_train = tf.one_hot(y_train, depth=10)
y_test = tf.one_hot(y_test, depth=10)

Training set으로부터 Validation set을 따로 분리합니다.

In [0]:
x_train, x_val = x_train[:50000], x_train[50000:]
y_train, y_val = y_train[:50000], y_train[50000:]

In [0]:
print(x_train.shape)
print(x_val.shape)
print(x_test.shape)

print(y_train.shape)
print(y_val.shape)
print(y_test.shape)


tf.data를 사용하여 데이터셋을 섞고 배치를 만듭니다:

In [0]:
train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(len(x_train)).batch(32) 
val_ds = tf.data.Dataset.from_tensor_slices((x_val, y_val)).batch(32) 
test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32)

케라스(Keras)의 [모델 서브클래싱(subclassing) API](https://www.tensorflow.org/guide/keras#model_subclassing)를 사용하여 `tf.keras` 모델을 만듭니다:

In [0]:
class MyModel(Model):
  def __init__(self):
    super(MyModel, self).__init__()
    self.flatten = Flatten()
    self.hidden_1 = Dense(50, activation='sigmoid')
    self.output_layer = Dense(10, activation='softmax')

  def call(self, x):
    x = self.flatten(x)
    x = self.hidden_1(x)  
    return self.output_layer(x)

model = MyModel()

훈련에 필요한 옵티마이저(optimizer)와 손실 함수를 선택합니다.

In [0]:
loss_object = tf.keras.losses.MeanSquaredError()

optimizer = tf.keras.optimizers.SGD(learning_rate=1.0)

모델의 손실과 성능을 측정할 지표를 선택합니다. 에포크가 진행되는 동안 수집된 측정 지표를 바탕으로 최종 결과를 출력합니다.

In [0]:
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.CategoricalAccuracy(name='train_accuracy')

val_loss = tf.keras.metrics.Mean(name='val_loss')
val_accuracy = tf.keras.metrics.CategoricalAccuracy(name='val_accuracy')
 
test_accuracy = tf.keras.metrics.CategoricalAccuracy(name='test_accuracy')

`tf.GradientTape`를 사용하여 모델을 훈련합니다:

In [0]:
@tf.function
def train_step(images, labels):
  with tf.GradientTape() as tape:
    predictions = model(images) 
    loss = loss_object(labels, predictions)
  gradients = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(gradients, model.trainable_variables))

  train_loss(loss)
  train_accuracy(labels, predictions)

이제 모델을 20 Epoch 동안 학습하면서, 테스트할 예정입니다.

In [0]:
@tf.function
def val_step(images, labels):
  predictions = model(images)
  v_loss = loss_object(labels, predictions)

  val_loss(v_loss)
  val_accuracy(labels, predictions)

In [0]:
@tf.function
def test_step(images, labels):
  predictions = model(images) 
  test_accuracy(labels, predictions)

In [0]:
train_ACC = []
train_LOSS = []
val_ACC = []
val_LOSS = [] 

EPOCHS = 20

for epoch in range(EPOCHS):
  for images, labels in train_ds:
    train_step(images, labels)

  for val_images, val_labels in val_ds:
    val_step(val_images, val_labels)

  template = 'Epoch: {}, Loss: {}, ACC: {}, Validation Loss: {}, Validation ACC: {}'
  print (template.format(epoch+1,
                         train_loss.result(),
                         train_accuracy.result()*100,
                         val_loss.result(),
                         val_accuracy.result()*100))
  
  train_ACC.append(train_accuracy.result()*100)
  train_LOSS.append(train_loss.result())
  val_ACC.append(val_accuracy.result()*100)
  val_LOSS.append(val_loss.result())

for test_images, test_labels in test_ds:
  test_step(test_images, test_labels)
 
print ('Test ACC: {}'.format(test_accuracy.result()*100)) 

위와 같은 설정을 따라했다면 validation set과 test set에서 약 92%의 테스트 정확도를 달성합니다.


아래에는 학습과정 동안의 Train/Validation Accuracy, Loss를 그래프로 확인할 수 있습니다.

In [0]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10,10))
plt.plot(range(1,EPOCHS+1), train_ACC, label="Train")
plt.plot(range(1,EPOCHS+1), val_ACC, label="Validation")

plt.title("Accuracy Graph")
plt.xticks(range(1,EPOCHS+1), range(1,EPOCHS+1))
plt.legend(loc=2)

plt.show()

In [0]:
plt.figure(figsize=(10,10))
plt.plot(range(1,EPOCHS+1), train_LOSS, label="Train")
plt.plot(range(1,EPOCHS+1), val_LOSS, label="Validation")

plt.title("Loss Graph")
plt.xticks(range(1,EPOCHS+1), range(1,EPOCHS+1))
plt.legend(loc=3)

plt.show()

# 실습 MISSION #3

* 위 Learning curve로부터 확인할 수 있는 현상은 무엇인가? 더 나은 성능을 내기 위해 네트워크를 수정해보자!

# 실습 MISSION #4

다음과 같이 네트워크를 수정하고 그래프를 그려보자 
* 두 번째 레이어를 추가하여 hidden size를 40으로 설정
* 세 번째 레이어를 추가하여 hidden size를 30으로 설정
* learning rate를 10.0으로 설정
* 50 epoch동안 학습

무엇이 문제인가?